In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("yitingxie/rlhf-reward-datasets", split=["test"])


In [ ]:
raw_dataset = list(raw_dataset[0])
raw_dataset[:3]


In [ ]:
from typing import Mapping
from llama_likes import Completion, PreferenceInput


def to_preference_input(entry: Mapping[str, str]) -> PreferenceInput:
    def clean(string: str) -> str:
        string = string.strip()
        if string.startswith("Human: "):
            return string.replace("Human: ", "", 1)
        return string.replace("Assistant: ", "", 1)
    return PreferenceInput(
        instruction=clean(entry["prompt"]),
        completion_a=Completion(player_id="chosen", completion=clean(entry["chosen"])),
        completion_b=Completion(player_id="rejected", completion=clean(entry["rejected"])),
    )


data: list[PreferenceInput] = [to_preference_input(d) for d in raw_dataset]
data[:3]


Labeling a few examples with GPT4...

In [ ]:
from typing import Union
from llama_likes import OpenaiModel, OpenaiRanker, PreferenceResult, PreferenceError

openai_model = OpenaiModel.GPT_4
openai_ranker = OpenaiRanker(openai_model)

labels: list[Union[PreferenceResult, PreferenceError]] = []
for input in data[:5]:
    label = openai_ranker.rank(input)
    labels.append(label)

labels
